In [0]:
from numpy import asarray
import numpy as np
import pickle
import keras
import warnings
import sklearn.model_selection as sk
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from keras.layers.pooling import MaxPooling2D
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib  inline
from os import listdir
from matplotlib import image
import random
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from keras.layers import Reshape
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from matplotlib import pyplot 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd "/content/drive/My Drive/BanglaLekha/images"
%ls

In [0]:
path  = "1/"
DIV_1 = []
DIV_2 = []
for r, d, file in os.walk(path):
    for i in file:
        print(i)
        if i[0:2]=="01":
          DIV_1.append(cv2.imread(os.path.join(path,i),cv2.IMREAD_COLOR))
        else:
          DIV_2.append(cv2.imread(os.path.join(path,i),cv2.IMREAD_COLOR))

In [0]:
Re_DIV1 = []
Re_DIV2 = []
IMG_SIZE = 128
for i in range(len(DIV_1)):
  resized = cv2.resize(DIV_1[i],(IMG_SIZE,IMG_SIZE))
  Re_DIV1.append(resized)
  
for i in range(len(DIV_2)):
  resized1 = cv2.resize(DIV_2[i],(IMG_SIZE,IMG_SIZE))
  Re_DIV2.append(resized1)

In [0]:
plt.imshow(Re_DIV1[85])
plt.show()

In [0]:
train = []
for i in Re_DIV1:
  train.append([i,0])
  
for i in Re_DIV2:
    train.append([i,1])

random.shuffle(train)
for batch in train[0:50]:
    print(batch[1])

In [0]:
X_train =[]
Y_train =[]

for f,l in train:
    X_train.append(f)
    Y_train.append(l)

In [0]:
IMG_SIZE = 128

X_train = np.array(X_train).reshape(-1,IMG_SIZE,IMG_SIZE,3)

In [0]:
X_train.shape

(1977, 128, 128, 3)

In [0]:
temp_test = Y_train.copy()
temp_test  = np.array(temp_test)
temp_test.shape

(1977,)

In [0]:
from keras.applications.vgg16 import VGG16
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(128,128,3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [0]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png') 
	pyplot.close()

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [0]:
X_train1, X_test1, y_train, y_test =  sk.train_test_split(X_train,temp_test,test_size=0.3, random_state = 42)

y_test.shape

(594,)

In [0]:
def run_test_harness():
	# define model
	model = define_model()
	history = model.fit(X_train1, y_train, batch_size=100, epochs=35,verbose = 1)
	filename = 'division_finalized_model.sav'
	pickle.dump(model, open(filename, 'wb'))	
	# evaluate model
	_, acc = model.evaluate(X_test1, y_test, verbose=1)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)

In [0]:
run_test_harness()

In [0]:
loaded_model = pickle.load(open('division_finalized_model.sav', 'rb'))
result = loaded_model.evaluate(X_test1, y_test)
print(result)

594/594 [==============================] - 1s 2ms/step
[6.17297751975782, 0.6127946125939238]
